# FHR Baseline from Wróbel et al. 2013

## Summary

This notebook implements the method for FHR baseline calculation from Wróbel et al. 2013, based on the MATLAB implementation in the FHRMA package.

The paper: [Wróbel et al. 2013](https://doi.org/10.1016/j.bbe.2013.09.004) - Improving fetal heart rate signal interpretation by application of myriad filtering

## Paper description of the method

This method uses myriad filtering, with the stated benefit being that it does not require interpolation of signal loss segments (although we have already done that for all FHR records here).



## Python implementation

Import required packages

In [1]:
import math
import numpy as np
import pandas as pd
import sys

Import functions defined elsewhere

In [2]:
sys.path.append('/functions')
from functions.load_data import load_data
from functions.avg_subsample import avg_subsample

Import datasets required for running the function and validation

In [3]:
raw_fhr, fhrma_base, fhrma_acc, fhrma_dec = load_data(fhrma_path = './data/W_std.mat')

Define functions for calculating baseline

In [4]:
def myriad(x, k2):
    '''
    Implements myriad filter, based on myriad() in FHRMA

    Parameters
    ----------
    x : array
        Subsample of sfhr
    k2 : numeric
        Not certain.
    '''
    # Get min and max of x, and create empty list for length of difference 
    # between the min and the max, plus 1
    ma = round(max(x))
    mi = round(min(x))
    myrs = np.zeros(ma-mi+1)

    # For values of i up to length of myrs, manipulate each value in x, then sum
    for i in np.arange(0, len(myrs)):
        myrs[i] = sum([math.log(k2+((f-i-mi)**2)) for f in x])

    # Get index of minimum
    m = np.argmin(myrs)

    # Add that index to the minimum value of x (plus 1 as Python is 0-indexed)
    m = m + mi + 1

    return (m)

In [5]:
def wrobel_baseline(fhr):
    '''
    Calculate FHR baseline using method from Wrobel et al. 2013, as implemented
    in MATLAB in the FHRMA toolbox

    Parameters
    ----------
    fhr : array
        Cleaned FHR
    '''
    # Find mean of every 10 records, generating shorter version of FHR (sfhr)
    sfhr = avg_subsample(fhr, 10)

    # Create array of zeros with same length of sFHR
    baseline = np.zeros(len(sfhr))

    for i in np.arange(1, len(sfhr)+1):
        # wins=161 for i <= 161, and wins=i-160 for i > 162
        # wine=i+160 for i < len(sfhr)-160, and wine=len(sfhr) for i >= len(sfhr)-160
        # No -1 for wine as MATLAB includes the final index but Python wouldn't
        wins = max(1, i-160)-1
        wine = min(len(sfhr), i+160)

        # Filter to that interval in sfhr, and set value of k2
        x = sfhr[wins:wine]
        k2 = 0.25

        # Set baseline to outcome of myriad function
        baseline[i-1] = myriad(x, k2)

    # Create array of zeros with length of original FHR
    full_baseline = np.zeros(len(fhr))

    # Get the index of each item in baseline, and find equivalent for full sequence
    indices = np.arange(0, len(baseline))*10
    indices[1:]-= 1

    # Set values
    full_baseline[indices] = baseline

    # Perform linear interpolation on the values in between (and not past them)
    max_index = (max(indices))+1
    full_baseline[:max_index][full_baseline[:max_index] == 0] = np.nan
    full_baseline = pd.Series(full_baseline).interpolate(method='linear').values

    return(full_baseline)

Compare a few examples against fhrma

In [6]:
python_result = wrobel_baseline(raw_fhr['train28'])
fhrma_result = fhrma_base['train28']
pd.Series(python_result == fhrma_result).value_counts()

True    17595
Name: count, dtype: int64

In [7]:
python_result = wrobel_baseline(raw_fhr['test01'])
fhrma_result = fhrma_base['test01']
pd.Series(python_result == fhrma_result).value_counts()

True    24944
Name: count, dtype: int64